## Apriori
Apriori is all about people who bought this also bought other things.
The algorithm has three parts-
* support
* confidence
* lift

### Support-
Support(I) = # trnsaction containing I / # transactions

### Confidence -
confidence(I1 -> I2) = #transactions containing I1 and I2 / # transactions containing I1

### Lift-
lift(I1 -> I2) = confidence(I1 -> I2) / support(I2)

### Steps -
* Set a minimum support and confidence.
* Take all the subsets in the transactions having higher support than minimum support.
* Take all the rules of these subsets having higher confidence than minimum confidence.
* Sort the rules by decreasing lift.

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from apyori import apriori

In [67]:
dataset = pd.read_csv(r"/home/webhav/Documents/my_project_dir/MachineLearning/Part 5 - Association Rule Learning/Section 28 - Apriori/Apriori_Python/Market_Basket_Optimisation.csv",header = None)

In [68]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
dataset.shape

(7501, 20)

In [70]:
transactions=[]

In [71]:
for i in range(dataset.shape[0]):
    transactions.append([str(dataset.values[i,j]) for j in range(0,20)])

In [72]:
len(transactions)

7501

In [73]:
#calculating minimum_support to pass in apriory-
#we want a product to be sold atleast 3 times a day then in a week it will be 7*3 deviding wiith total transactions
minimum_support = round(3*7/7500,3)
(minimum_support)

0.003

In [74]:
#calculating confidence-
# we do not need too high or too low confidenc so taking 20 %.
minimum_confidence = 0.2

In [75]:
#minimum lift
minimum_lift = 3

In [76]:
rules = apriori(transactions,min_support = 0.003, min_confidence=0.2,min_lift=3,min_length=2)

In [77]:
results = list(rules)

In [86]:
lift = []
association = []
for i in range (0, len(results)):
    lift.append(results[:len(results)][i][2][0][3])
    association.append(list(results[:len(results)][i][0]))

In [87]:
rank = pd.DataFrame([association, lift]).T
rank.columns = ['Association', 'Lift']

In [88]:
# Show top 10 higher lift scores
rank.sort_values('Lift', ascending=False).head(10)

,Association,Lift
150,"[milk, nan, frozen vegetables, mineral water, ...",7.98718
97,"[soup, milk, mineral water, frozen vegetables]",7.98718
149,"[milk, nan, olive oil, frozen vegetables, mine...",6.12827
96,"[olive oil, milk, mineral water, frozen vegeta...",6.12827
132,"[olive oil, nan, mineral water, whole wheat pa...",6.12827
59,"[olive oil, whole wheat pasta, mineral water]",6.11586
50,"[spaghetti, ground beef, tomato sauce]",5.53597
122,"[spaghetti, nan, ground beef, tomato sauce]",5.53597
28,"[fromage blanc, nan, honey]",5.17882
3,"[fromage blanc, honey]",5.16427


# Eclat
Here we calculate the support but we create set of different items and calulate support betweeen them.

## steps-
* set a minimum support.
* Take all the subsets in the transactions having higher support then minimum.
* Sort these subsets by decreasing support.

In [90]:
# Putting all transactions in a single list
items = []
for i in range(0, len(transactions)):
    items.extend(transactions[i])

# Finding unique items from transactions and removing nan
uniqueItems = list(set(items))
uniqueItems.remove('nan')

In [91]:
# creating combinations of items in pairs-
pair = []
for j in range(0, len(uniqueItems)):
    k = 1;
    while k <= len(uniqueItems):
        try:
            pair.append([uniqueItems[j], uniqueItems[j+k]])
        except IndexError:
            pass
        k = k + 1;

### Calculating score
The calculation is done looking at the number of customers that bought both items (the pair) and divided by all customers of the week (7501). This calculation is done for all pairs possible and the score is returned on "score" list.

In [92]:
score = []
for i in pair:
    cond = []
    for item in i:
        cond.append('("%s") in s' %item)
    mycode = ('[s for s in transactions if ' + ' and '.join(cond) + ']')
    #mycode = "print 'hello world'"
    score.append(len(eval(mycode))/7501.)

In [93]:
ranking_ECLAT = pd.DataFrame([pair, score]).T
ranking_ECLAT.columns = ['Pair', 'Score']

In [94]:
ranking_ECLAT.sort_values('Score', ascending=False).head(10)

,Pair,Score
4664,"[spaghetti, mineral water]",0.0597254
974,"[chocolate, mineral water]",0.0526596
3074,"[eggs, mineral water]",0.0509265
4868,"[milk, mineral water]",0.0479936
3676,"[ground beef, mineral water]",0.0409279
3666,"[ground beef, spaghetti]",0.0391948
964,"[chocolate, spaghetti]",0.0391948
3064,"[eggs, spaghetti]",0.0365285
3073,"[eggs, french fries]",0.0363951
5319,"[mineral water, frozen vegetables]",0.0357286
